#Silver Layer – Data Cleaning & Standardization

## Reading From Bronze Table


In [0]:
df=spark.table("airbnb.bronze.listings_table")


##Data Transformation

In [0]:
#displaying the dataframe for exploratrion 
df.display()

In [0]:
#importing pyspark functions

from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F


In [0]:
#changing schema datatype for columns and set invalid values to null 
from pyspark.sql import functions as F

df = df.withColumn("id", F.expr("try_cast(id as decimal(38,0))")) \
       .withColumn("host_id", F.expr("try_cast(host_id as int)")) \
       .withColumn("latitude", F.expr("try_cast(latitude as double)")) \
       .withColumn("longitude", F.expr("try_cast(longitude as double)")) \
       .withColumn("price", F.expr("try_cast(price as double)")) \
       .withColumn("minimum_nights", F.expr("try_cast(minimum_nights as int)")) \
       .withColumn("number_of_reviews", F.expr("try_cast(number_of_reviews as int)")) \
       .withColumn("last_review", F.expr("try_cast(last_review as date)")) \
       .withColumn("reviews_per_month", F.expr("try_cast(reviews_per_month as double)")) \
       .withColumn("calculated_host_listings_count", F.expr("try_cast(calculated_host_listings_count as int)")) \
       .withColumn("availability_365", F.expr("try_cast(availability_365 as int)")) \
       .withColumn("number_of_reviews_ltm", F.expr("try_cast(number_of_reviews_ltm as int)"))

    



In [0]:
df.display()

In [0]:
#trim any white spaces for each string column data
for field in df.schema.fields:
  if field.dataType == 'string':
    df = df.withColumn(field.name, F.trim(F.col(field.name)))


In [0]:
#droping unessary columns
df = df.drop("host_name","neighbourhood_group","license")


In [0]:
df.display()

In [0]:
#Drop duplicates in id column
df=df.dropDuplicates(['id'])
  

In [0]:
df.display()

In [0]:
# removing null value from host_id column and id using filter

df = df.filter(df.id.isNotNull() & df.host_id.isNotNull())
df.count()
#

In [0]:
df.display()


In [0]:
#drop prices with null values
df = df.filter(df.price.isNotNull())
df.display()

In [0]:
#fill null values with 0 in columns
df = df.fillna(0,['minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365','number_of_reviews_ltm'])



In [0]:
df.display()

##Write Into Silver Table

In [0]:
#writing data into a silver layer as table
df.write.mode("overwrite").format("delta").saveAsTable("airbnb.silver.listing_table")
